In [17]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

# Data Science Level 2 Practical Problem 2305-2

In [2]:
set1 = pd.read_csv('lv2-2305-2.csv')

In [3]:
set1

,REG_NO,GRE,TOEFL,UNIV_RATING,SOP,LOR,CGPA,RESEARCH,ADMIT
0,1,337,118,B,4.5,4.5,9.65,1,Admitted
1,2,324,107,B,4.0,4.5,8.87,1,Admitted
2,3,316,104,C,3.0,3.5,8.00,1,Denied
3,4,322,110,C,3.5,2.5,8.67,1,Admitted
4,5,314,103,D,2.0,3.0,8.21,0,Denied
...,...,...,...,...,...,...,...,...,...
395,396,324,110,C,3.5,3.5,9.04,1,Admitted
396,397,325,67,C,3.0,3.5,9.11,1,Admitted
397,398,330,116,B,5.0,4.5,9.45,1,Admitted
398,399,312,103,C,3.5,4.0,8.78,0,Denied


In [4]:
set1.shape

(400, 9)

In [5]:
set1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
REG_NO         400 non-null int64
GRE            400 non-null int64
TOEFL          400 non-null int64
UNIV_RATING    400 non-null object
SOP            400 non-null float64
LOR            400 non-null float64
CGPA           400 non-null float64
RESEARCH       400 non-null int64
ADMIT          400 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 28.2+ KB


In [6]:
set1.describe()

,REG_NO,GRE,TOEFL,SOP,LOR,CGPA,RESEARCH
count,400.000000,400.000000,400.00000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,106.96000,3.400000,3.452500,8.598925,0.547500
std,115.614301,11.473646,7.82089,1.006869,0.898478,0.596317,0.498362
min,1.000000,290.000000,52.00000,1.000000,1.000000,6.800000,0.000000
25%,100.750000,308.000000,103.00000,2.500000,3.000000,8.170000,0.000000
50%,200.500000,317.000000,107.00000,3.500000,3.500000,8.610000,1.000000
75%,300.250000,325.000000,112.00000,4.000000,4.000000,9.062500,1.000000
max,400.000000,340.000000,120.00000,5.000000,5.000000,9.920000,1.000000


### Q1

In [15]:
### Q1.
df1 = set1.copy()

df1_a = df1[df1['ADMIT']=='Admitted']
df1_d = df1[df1['ADMIT']=='Denied']
print('PASS rows: ', df1_a.shape)
print('FAIL rows: ', df1_d.shape)

PASS rows:  (180, 9)
FAIL rows:  (220, 9)


In [18]:
var1=['GRE','TOEFL','CGPA']
for i in var1:
    df1_a[i+'_R']=df1_a[i].rank(method='min',ascending=False)
    df1_d[i+'_R']=df1_d[i].rank(method='min',ascending=False)

In [19]:
# Step 1-3.
var1_s = ['GRE_R','TOEFL_R','CGPA_R']
result1_3 = df1_a[var1_s].corr(method='spearman')['CGPA_R']
print(result1_3)
print('##############################')
A_VAR = result1_3.sort_values().index[1]
A= result1_3.sort_values()[1]
print('A_VAR: ', A_VAR)
print('A: ', A)

GRE_R      0.777091
TOEFL_R    0.738595
CGPA_R     1.000000
Name: CGPA_R, dtype: float64
##############################
A_VAR:  GRE_R
A:  0.7770913247704572


In [20]:
# Step 1-4.
result1_4 = df1_d[var1_s].corr(method='spearman')['CGPA_R']
print(result1_4)
print('##############################')
B_VAR = result1_4.sort_values().index[1]
B= result1_4.sort_values()[1]
print('B_VAR: ', B_VAR)
print('B: ', B)

GRE_R      0.595403
TOEFL_R    0.616724
CGPA_R     1.000000
Name: CGPA_R, dtype: float64
##############################
B_VAR:  TOEFL_R
B:  0.616724306652575


In [21]:
print('A: ', A)
print('B: ', B)
A21=(A+B)/2 if A_VAR==B_VAR else abs(A-B)
print ('Q1 Answer: ', A21)

A:  0.7770913247704572
B:  0.616724306652575
Q1 Answer:  0.16036701811788212


### Q2

In [22]:
### Q2.
df2 = set1.copy()

# Step 2-1.
q1 = df2['TOEFL'].quantile(0.25)
q3 = df2['TOEFL'].quantile(0.75)
iqr = q3-q1

df2['outlier']=df2['TOEFL'].apply(lambda x : 1 if (x<q1-1.5*iqr)|(x>q3+1.5*iqr) else 0)
df2=df2[df2['outlier']==0].drop('outlier',axis=1)

df2.shape

(396, 9)

In [23]:
# Step 2-2.
df2['ADMIT_S']= df2['ADMIT'].apply(lambda x : 1 if x=='Admitted' else 0)

In [24]:
# Step 2-3.
var2_s = ['GRE','TOEFL','SOP','LOR','CGPA']
df2_s = df2[var2_s]
y_train = df2['ADMIT_S']
from sklearn.linear_model import LogisticRegression
predict2 = LogisticRegression(solver = 'newton-cg', C=100000, random_state=1234).fit(df2_s,y_train).predict(df2_s)

In [25]:
# Step 2-4.
from sklearn.metrics import accuracy_score
A22 = accuracy_score(y_train,predict2)*100
print('Q2 Answer: ', A22)

Q2 Answer:  87.62626262626263


### Q3

In [26]:
##### Q3
df3 = set1.copy()

# Step 3-1.
df3 = df3[df3['ADMIT']=='Denied']
#Step 3-2.
dummies3 = pd.get_dummies(df3['UNIV_RATING'], drop_first=True).reset_index(drop=True)

In [27]:
#Step 3-3.
var3=['GRE','TOEFL','SOP','LOR','CGPA']

from sklearn.preprocessing import MinMaxScaler
df3_s = pd.DataFrame(MinMaxScaler().fit_transform(df3[var3]), columns=var3)

In [28]:
#Step 3-4.
train3 = pd.concat([df3_s, df3['RESEARCH'].reset_index(drop=True),dummies3], axis=1)
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [29]:
# cluster 4
predict3_4 = KMeans(random_state=1234, n_clusters=4, n_init = 50, max_iter = 300).fit(train3).predict(train3).tolist()
silhouette_score(train3, predict3_4)

0.49105088711202427

In [30]:
# cluster 5
predict3_5 = KMeans(random_state=1234, n_clusters=5, n_init = 50, max_iter = 300).fit(train3).predict(train3).tolist()
silhouette_score(train3, predict3_5)

0.5090779061903452

In [31]:
# cluster 6
predict3_6 = KMeans(random_state=1234, n_clusters=6, n_init = 50, max_iter = 300).fit(train3).predict(train3).tolist()
silhouette_score(train3, predict3_6)

0.5556812882602017

In [37]:
unbalance5 = predict3_5.count(max(predict3_5, key=predict3_5.count))/predict3_5.count(min(predict3_5, key=predict3_5.count))
print('cluster 5 IR: ', unbalance5)

cluster 5 IR:  3.0416666666666665


In [38]:
unbalance6 = predict3_6.count(max(predict3_6, key=predict3_6.count))/predict3_6.count(min(predict3_6, key=predict3_6.count))
print('cluster 6 IR: ', unbalance6)

cluster 6 IR:  5.615384615384615


In [39]:
C= 5
D= silhouette_score(train3, predict3_5)
A23 = C+D
print('Q3 Answer: ', A23)

Q3 Answer:  5.509077906190345
